In [ ]:
from fast_bert.data_abs import BertAbsDataBunch
from fast_bert.learner_abs import BertAbsLearner
from box import Box
import logging
import torch
from pathlib import Path
from transformers import BertTokenizer

In [ ]:
from tokenizers import (ByteLevelBPETokenizer,
                            BPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)

In [ ]:
PATH = Path("../../summarisation/")
DATA_PATH = PATH/'data'
MODEL_PATH = PATH/'model'

In [ ]:
logger = logging.getLogger()

In [ ]:
args = Box({
    "max_seq_length": 512,
    "batch_size": 8,
    "learning_rate": 5e-3,
    "num_train_epochs": 6,
    "fp16": True,
    "model_name": 'bertabs-finetuned-cnndm',
    "model_type": 'bert'
})

In [ ]:
device = torch.device('cuda') if torch.cuda.device_count() else torch.device('cpu')
if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

In [ ]:
tokenizer = BertWordPieceTokenizer(str(MODEL_PATH/'vocab.txt'), lowercase=True)

In [ ]:
databunch = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=tokenizer, device=device)
databunch_old = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=args.model_name, device=device)

In [ ]:
databunch_with_data = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=args.model_name, device=device)

In [ ]:
databunch_with_data_new_tokeniser = BertAbsDataBunch(data_dir=DATA_PATH, tokenizer=tokenizer, device=device)

In [ ]:
learner = BertAbsLearner.from_pretrained_model(databunch, MODEL_PATH, device, logger=logger)

In [ ]:
texts = databunch_with_data.test_dl.dataset[0][1]
texts

In [ ]:
%%timeit
learner.predict_batch(texts)

In [ ]:
learner_old = BertAbsLearner.from_pretrained_model(databunch_old, MODEL_PATH, device, logger=logger)

In [ ]:
%%timeit
learner_old.predict_batch(texts)